<a href="https://colab.research.google.com/github/IzhanAli08/Machine-Learning-Projects/blob/main/CarPricePredictionProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Car Price Prediction using Regression

In this project, the goal is to predict car prices based on inputs/specifications of a car. For example, the model that we build, should be able to correctly tell the price of a car if provided with model, make, year, mileage, and other important characteristics of a car. This is an important problem for many car resale businesses.

For any problem that requires the prediction of a continuous value, we use something known as regression. I read about regression here:

[Linear Regression](https://www.analyticsvidhya.com/blog/2021/10/everything-you-need-to-know-about-linear-regression/)

Before starting with the project, make sure you understand the idea of linear regression.

### Project Plan

The plan for the project is the following:

1 First, we download the dataset.

2 Next, we do some preliminary analysis of the data. This is called Exploratory data analysis(EDA).

3 After that, we set up a validation strategy to make sure our model produces correct
predictions. This involves testing model performance on unknown data points.

4 Then we implement a linear regression model in Python and NumPy.

5 Next, we cover feature engineering to extract important features from the data
to improve the model.

6 Finally, we see how to make our model stable with regularization and use it to
predict car prices.

#### References

Machine learning Bootcamp (Grigov)

https://numpy.org/

